# Forecast Electric power consumption - Using ARIMA Model

## Table of Content
<ul>
    <li><a href="#arima1">Modeling with ARIMA Befor Augmenting</a></li>
    <li><a href="#arima2">Modeling with ARIMA After Augmenting</a></li>
</ul>

In [13]:
import pandas as pd
import math
import numpy as np
# ---------------------------------------

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
# ---------------------------------------

from prophet import Prophet
import holidays
from sklearn.metrics import mean_squared_error, mean_absolute_error
# ---------------------------------------

from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

In [11]:
def calc_order(df, train_len, steps,p_values, d_values, q_values) :
    p_ret = 0
    d_ret = 0
    q_ret = 0
    rmse = 1000
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order=(p,d,q)
                train = df[:train_len]
                test = df[train_len:]
                model=ARIMA(train,order = (p,d,q))
                model_fit=model.fit()
                y_pred=model_fit.forecast(steps=steps)
                error=np.sqrt(mean_squared_error(test,y_pred))
                if error < rmse and error >= 0:
                    rmse = error
                    p_ret = p
                    d_ret = d
                    q_ret = q
    return [p_ret, d_ret, q_ret]

#--------------------------------------------

def plot_train_test(df,col,train_len):
    train = df[:train_len]
    test = df[train_len:]
    
    train_samples = go.Scatter(x = train.index,
                  y = train[col],
                  mode = 'lines',
                  name = 'Train')

    test_samples = go.Scatter(x = test.index,
                  y = test[col],
                  mode = 'lines',
                  name = 'Test')

    layout = go.Layout(title={'text': 'Train/test split',
                          'y':0.9,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'},
                   xaxis = dict(title = 'Datetime'),
                   yaxis = dict(title = 'current'),
                   template = 'plotly_dark')

    data = [train_samples, test_samples]
    fig = go.Figure(data = data, layout = layout)
    iplot(fig)

#--------------------------------------------

def arima_modeling(df, col, train_len, steps):
    p , d , q = calc_order(df, train_len, steps, (0, 4), (0, 3), (0, 4))
    data_train = df[:train_len]
    data_test = df[train_len:]
    data_model = ARIMA(data_train, order=(p, d, q))
    data_model_fit = data_model.fit()
    data_forecast=data_model_fit.get_forecast(steps=steps)
    MAPE(data_test[col].tolist(),data_forecast['mean'].tolist())
    forecast = data_model_fit.get_forecast(steps).summary_frame()
    
    df2 = pd.DataFrame({
        'Date' : pd.to_datetime(data_test.index),
        'mean' : forecast['mean'],
        'mean_se' : forecast['mean_se'],
        'mean_ci_lower' : forecast['mean_ci_lower'],
        'mean_ci_upper' : forecast['mean_ci_upper']
    })
    df2.set_index('Date', inplace=True)
   # print(data_train)
    #print(data_test)
    #print(df2)
    forecast_mean = data_forecast.predicted_mean
    forecast_ci = data_forecast.conf_int()
    
    # Plotting
    plt.figure(figsize=(20, 12))
    plt.title("ARIMA Model - Train, Test, and Forecast")
    
    # Plot training data
    plt.plot(pd.to_datetime(data_train.index), data_train[col], label='Training Data', color='blue')
    plt.plot(pd.to_datetime(data_test.index), data_test[col], label='Training Data', color='red')
    
    # Plot forecast mean
    plt.plot(pd.to_datetime(data_test.index), forecast_mean, color="darkgreen", label='Forecast')
    
    # Plot confidence intervals
    plt.fill_between(data_test.index, 
                     forecast_ci.iloc[:, 0], 
                     forecast_ci.iloc[:, 1], 
                     color='k', alpha=0.2, label='Confidence Interval')
    
    plt.legend()
    plt.show()
#--------------------------------------------

def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

<a id='arima1'></a>
## Modeling with Arima Before Augmenting
###  9 months train / 3 months test

In [12]:
# Arima model with 2019 Data 9 months train phase 1

df = pd.read_csv('input_data2019.csv')
df = df.dropna()

df_i1 = pd.DataFrame({
    'Datetime': df['Datetime'],
    'I1': df['I1']
});

df_i2 = pd.DataFrame({
    'Datetime': df['Datetime'],
    'I2': df['I2']
});
df_i3 = pd.DataFrame({
    'Datetime': df['Datetime'],
    'I3': df['I3']
});

df_i3.set_index('Datetime', inplace=True)

df_i2.set_index('Datetime', inplace=True)

df_i1.set_index('Datetime', inplace=True)

df.set_index('Datetime', inplace=True)

plot_train_test(df, 'I1', 6546)

print("Phase 1 modeling with Arima Model 9 months training / 3 months testing")
arima_modeling(df_i1, 'I1', 6546, 2207)

print("Phase 2 modeling with Arima Model 9 months training / 3 months testing")
arima_modeling(df_i2, 'I2', 6546, 2207)

print("Phase 3 modeling with Arima Model 9 months training / 3 months testing")
arima_modeling(df_i3, 'I3', 6546, 2207)

Phase 1 modeling with Arima Model 9 months training / 3 months testing


TypeError: 'PredictionResultsWrapper' object is not subscriptable

###  10 months train / 2 months test

In [ ]:
plot_train_test(df, 'I1', 7290)

print("Phase 1 modeling with Arima Model 10 months training / 2 months testing ")
arima_modeling(df_i1, 'I1', 7290, 1463)

print("Phase 2 modeling with Arima Model 10 months training / 2 months testing")
arima_modeling(df_i2, 'I2', 7290, 1463)

print("Phase 3 modeling with Arima Model 10 months training / 2 months testing")
arima_modeling(df_i3, 'I3', 7290, 1463)

### 11 months train / 1 month test

In [ ]:
plot_train_test(df, 'I1', 8010)

print("Phase 1 modeling with Arima Model 11 months training / 1 month testing")
arima_modeling(df_i1, 'I1', 8010, 743)

print("Phase 2 modeling with Arima Model 11 months training / 1 month testing")
arima_modeling(df_i2, 'I2', 8010, 743)

print("Phase 3 modeling with Arima Model 11 months training / 1 month testing")
arima_modeling(df_i3, 'I3', 8010, 743)

<a id='arima2'></a>
## Modeling with Arima after Augmenting
### 36 months train / 12 months test

In [ ]:
# Arima model with 2019 Data 9 months train phase 1

df = pd.read_csv('input_data2019-augmented.csv')
df = df.dropna()

df_i1 = pd.DataFrame({
    'Datetime': df['Datetime'],
    'I1': df['I1']
});

df_i2 = pd.DataFrame({
    'Datetime': df['Datetime'],
    'I2': df['I2']
});
df_i3 = pd.DataFrame({
    'Datetime': df['Datetime'],
    'I3': df['I3']
});

df_i3.set_index('Datetime', inplace=True)

df_i2.set_index('Datetime', inplace=True)

df_i1.set_index('Datetime', inplace=True)

df.set_index('Datetime', inplace=True)

plot_train_test(df, 'I1', 26305)

print("Phase 1 modeling with Arima Model 36 months training / 12 months testing")
arima_modeling(df_i1, 'I1', 26305, 9095)

print("Phase 2 modeling with Arima Model 36 months training / 12 months testing")
arima_modeling(df_i2, 'I2', 26305, 9095)

print("Phase 3 modeling with Arima Model 36 months training / 12 months testing")
arima_modeling(df_i3, 'I3', 26305, 9095)


### 42 months train / 6 months test

In [ ]:
plot_train_test(df, 'I1', 30649)

print("Phase 1 modeling with Arima Model 42 months training / 6 months testing")
arima_modeling(df_i1, 'I1', 30649, 4751)

print("Phase 2 modeling with Arima Model 42 months training / 6 months testing")
arima_modeling(df_i2, 'I2', 30649, 4751)

print("Phase 3 modeling with Arima Model 42 months training / 6 months testing")
arima_modeling(df_i3, 'I3', 30649, 4751)


### 45 months train / 3 months test

In [ ]:
plot_train_test(df, 'I1', 32857)

print("Phase 1 modeling with Arima Model 45 months training / 3 months testing")
arima_modeling(df_i1, 'I1', 32857, 2543)

print("Phase 2 modeling with Arima Model 45 months training / 3 months testing")
arima_modeling(df_i2, 'I2', 32857, 2543)

print("Phase 3 modeling with Arima Model 45 months training / 3 months testing")
arima_modeling(df_i3, 'I3', 32857, 2543)